# YT Comments analysis

In [44]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from datetime import datetime, timedelta, time
import json
import re
from api import API_KEY

channel_id = "UCWeg2Pkate69NFdBeuRFTAw" #Squeezie channel
etoiles = 'UCABf02qOye7XYapcK1M45LQ'
youtube = build('youtube', 'v3', developerKey=API_KEY)
exemple_video = "qCKyRhkhqoQ"
otp_recap = 'F7A8OCdmZ90'

### Class request
Class to handle youtube request since youtube api doesn't provide a request object

In [45]:
class Request:
    """ Class Request handling youtube request better """
    def __init__(self, requestType,part=None, id=None, chart=None, regionCode=None, maxResults=None, pageToken=None, videoId=None):
        self.requestType = requestType
        self.part = part
        self.id = id
        self.chart = chart
        self.regionCode = regionCode
        self.maxResults = maxResults
        self.pageToken = pageToken
        self.videoId = videoId
        
    def execute(self):
        param = vars(self) # Fetch class attributes
        param = {x:y for x,y in list(param.items())[1:] if y} # Delete requestType ([1:]) and None attributes
        
        request = self.requestType.list(**param)
        return request.execute()

### Decorator
Decorator to retry when youtube request fails (mostly due to timeout erros)

In [46]:
def retry_on_exception(max_attempts=5):
    def decorator(func):
        def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < max_attempts:
                try:
                    result = func(*args, **kwargs)
                except Exception as e:
                    attempts += 1
                    if attempts == max_attempts:
                        return []
                        raise  # Relancer l'exception si le nombre maximal de tentatives est atteint
                    else:
                        print(f"{attempts}: Une exception s'est produite : {e}")
                else:
                    return result  # Retourner le résultat si aucune exception n'est levée
                time.sleep(0.5)
        return wrapper
    return decorator

### Datetime convertions
Functions to convert iso formated date found in youtube api responses in datetime objects.

In [47]:
def iso_toDatetime(iso_date:str):
    """Converts an ISO 8601 formatted date to a datetime object."""
    return datetime.strptime(iso_date[:-1], '%Y-%m-%dT%H:%M:%S')

def datetime_toISO(dt_obj:datetime):
    """Converts a datetime object to an ISO 8601 formatted date."""
    return dt_obj.isoformat()[:-7]  # remove microseconds

def iso_toDelta(iso_duration:str):
    """Converts an ISO 8601 formatted duration to a timedelta object."""
    match = re.match(r'PT(\d+D)*(\d+H)*(\d+M)*(\d+S)', iso_duration)
    days, hours, minutes, seconds = [int(x[:-1]) if x else 0 for x in match.groups()]
    return timedelta(days=days,hours=hours, minutes=minutes, seconds=seconds)

def delta_toISO(delta_obj:timedelta):
    """Converts a timedelta object to an ISO 8601 formatted duration."""
    hours = delta_obj.seconds // 3600
    minutes = (delta_obj.seconds % 3600) // 60
    seconds = delta_obj.seconds % 60
    
    daysStr = f"{delta_obj.days}D" if delta_obj.days != 0 else ""
    hoursStr = f"{hours}H" if hours != 0 else ""
    minutesStr = f"{minutes}M" if minutes != 0 else ""
    secondsStr = f"{seconds}S" if seconds != 0 else ""
    return f"PT{daysStr}{hoursStr}{minutesStr}{secondsStr}"

# print(iso_toDelta('PT4D3H20M9S'))
# print(delta_toISO(iso_toDelta('PT20M9S')))

### Fectching functions
Functions to fetch channels, comments ant top vids infos.

In [48]:
def format_channel_data(channel_data: dict) -> dict[str|dict]:
    """ Structure raw channel data """
    data = {
        "channel_name": channel_data.get('snippet', {}).get('title'),
        "channel_id": channel_data.get('id'),
        "country": channel_data.get('snippet', {}).get('country',""),
        "stats": channel_data.get('statistics'),
        "topics": [wikilink.split('/')[-1] for wikilink in channel_data.get('topicDetails', {}).get('topicCategories', [])],
    }
    del data['stats']['hiddenSubscriberCount']
    return data

In [49]:
def get_channel_data(youtube, channel_id:str) -> dict[str|dict]:
    """ Request (by id) for most important channel stats """
    request = Request(
        requestType=youtube.channels(),
        part="snippet,contentDetails,statistics,topicDetails",
        id=channel_id
    )
    response = request.execute()
    rawData = response.get('items', [])[0]
    return format_channel_data(rawData)
    

get_channel_data(youtube, channel_id)

{'channel_name': 'SQUEEZIE',
 'channel_id': 'UCWeg2Pkate69NFdBeuRFTAw',
 'country': 'FR',
 'stats': {'viewCount': '10448305177',
  'subscriberCount': '18700000',
  'videoCount': '1624'},
 'topics': ['Television_program', 'Entertainment']}

In [50]:
def format_video_data(video_data: dict) -> dict[str|dict]:
    """ Structure raw video data """
    data = {
            "title": video_data.get('snippet', {}).get('title'),
            "id": video_data.get('id'),
            "publishedAt": video_data.get('snippet', {}).get('publishedAt'),
            "duration" : video_data.get('contentDetails').get('duration'),
            "ViewCount" : video_data.get('statistics', {}).get('viewCount'),
            "likeCount" : video_data.get('statistics', {}).get('likeCount'),
            "commentCount" : video_data.get('statistics', {}).get('commentCount'),  
            "tags" : video_data.get('snippet', {}).get('tags')
    }
    
    return data

# time = get_video_info(youtube, 'JRBGBjaR9Wg').get("publishedAt")
# print(datetime.strptime(time, '%Y-%m-%dT%H:%M:%SZ'))

In [51]:
def get_video_data(youtube, video_Id:str) -> dict[str|dict]:
    """ Request (by id) for most important video stats """
    request = Request(
        requestType=youtube.videos(),
        part="snippet,contentDetails,statistics,topicDetails",
        id=video_Id,
    )
    response = request.execute()
    
    rawData = response.get('items', [])[0]
    return format_video_data(rawData)

get_video_data(youtube, otp_recap)

{'title': "G2 trop forts pour l'Europe ? (OTP AfterLEC)",
 'id': 'F7A8OCdmZ90',
 'publishedAt': '2024-02-20T16:00:09Z',
 'duration': 'PT1H6M3S',
 'ViewCount': '20083',
 'likeCount': '391',
 'commentCount': '47',
 'tags': ['otp',
  'afterlec',
  'aftershow',
  'podcast',
  'league of legends',
  'lec',
  'g2',
  'winter']}

In [52]:
def get_Most_Popular_Video(youtube, region:str) -> list[dict]:
    """ Request for most populars videos stats """
    request = Request(
        requestType=youtube.videos(),
        part="snippet,contentDetails,statistics,topicDetails",
        chart="mostPopular",
        regionCode=region,
        maxResults=100,
        pageToken=''
    )
    response = request.execute()
    
    pages = [response]
    while response.get('nextPageToken'):
        request.pageToken = response.get('nextPageToken')
        response = request.execute()
        pages.append(response)
    
    top_videos = [format_video_data(videos) for page in pages for videos in page.get('items')]
    return top_videos

get_Most_Popular_Video(youtube, 'FR')

[{'title': "TIK TOK M'A FAIT ACHETER ÇA ! #6 (Jamais vu ça avant)",
  'id': '4j92PHFnp3g',
  'publishedAt': '2024-02-25T11:56:47Z',
  'duration': 'PT23M46S',
  'ViewCount': '2299474',
  'likeCount': '167012',
  'commentCount': '2277',
  'tags': ['Joyca',
   'video',
   'reaction',
   'Français',
   'tik tok',
   'objets tik tok']},
 {'title': 'CACHE-CACHE Dans Une ÉNORME VILLA En BELGIQUE ! (Feat. Les Croûtons)',
  'id': 'tRAOVffz9LA',
  'publishedAt': '2024-02-25T13:00:47Z',
  'duration': 'PT34M27S',
  'ViewCount': '2056849',
  'likeCount': '158940',
  'commentCount': '3020',
  'tags': ['Michou',
   'Inox',
   'Inoxtag',
   'Lebouseuh',
   'Valouzz',
   'Pidi',
   'Croûton',
   'Croutons',
   'Vacances',
   'Dobby',
   'Cache',
   'Cache-cache',
   'Vidéo']},
 {'title': 'On joue a STUMBLE GUYS avec Flamby / Byilhan / Helydia ! 21h réaction à mon Fan Film !',
  'id': '-PmR-1Vn9fU',
  'publishedAt': '2024-02-26T21:08:30Z',
  'duration': 'PT2H4M30S',
  'ViewCount': '400764',
  'likeCount

In [53]:
def format_comment_data(comment:dict) -> dict[str|dict]:
    """ Structure raw comment data """
    data = {
        "id": comment.get('id'),
        "comment": comment.get('snippet', {}).get('textOriginal'),
        # "viewerRating": comment.get('snippet', {}).get('viewerRating'),
        "likeCount": comment.get('snippet', {}).get('likeCount'),
        "publishedAt": comment.get('snippet', {}).get('publishedAt'),
        "updatedAt": comment.get('snippet', {}).get('updatedAt')
        }
    
    return data

def format_threadedComment_data(comment:dict) -> dict[str|dict]:
    """ Structure raw threaded comment data """
    data = {
        "id": comment.get('id'),
        "topLevelComment": format_comment_data(comment.get('snippet', {}).get('topLevelComment')),
        "totalReplyCount": comment.get('snippet', {}).get('totalReplyCount'),
        # "replies": [format_comment_data(com) for com in comment.get('replies', {}).get('comments', [])]
        }
    
    return data

In [54]:
def get_comment(youtube,comment_id:str) -> dict[str|dict]:
    """ Request (by id) for most important comment stats """
    request = Request(
        requestType=youtube.comments(),
        part="snippet,id",
        id=comment_id,
    )
    response = request.execute()
    # print(response)
    rawData = response.get('items')[0]
    return format_comment_data(rawData)

get_comment(youtube, 'UgwUQR2JJFJSkihWLhx4AaABAg')

{'id': 'UgwUQR2JJFJSkihWLhx4AaABAg',
 'comment': "Avez vous déjà rêvé de la vie de pirate dans votre enfance ? Dans ce cas Skull & Bones pourrait vous intéresser ! Plus d'infos ici :  https://ubi.li/1aYqD",
 'likeCount': 25,
 'publishedAt': '2024-02-08T21:54:35Z',
 'updatedAt': '2024-02-08T21:54:35Z'}

In [55]:
@retry_on_exception(max_attempts=3)
def get_video_commentThreads(youtube, video_Id:str, maxComments:int) -> dict[str|list]:
    """ Request (by id) for all comments of a videos """
    request = Request(
        requestType=youtube.commentThreads(),
        part="snippet,id,replies",
        videoId=video_Id,
        maxResults=100
        # pageToken = ''
    )
    response = request.execute()
    
    maxComments -= response.get('pageInfo', {}).get('totalResults')
    comments = [format_threadedComment_data(comments) for comments in response.get('items',{})]
    while response.get('nextPageToken'):
        request.pageToken = response.get('nextPageToken')
        # time.sleep(0.3)
        response = request.execute()
        comments += [format_threadedComment_data(comments) for comments in response.get('items',{})]
        if (maxComments:= maxComments - response.get('pageInfo', {}).get('totalResults')) <= 0:
            break
        
    print(f"Fetched {len(comments)} comments !")
    return comments

get_video_commentThreads(youtube, otp_recap, 1000)

Fetched 23 comments !


[{'id': 'Ugw6FJfQqKe0KrTgQF54AaABAg',
  'topLevelComment': {'id': 'Ugw6FJfQqKe0KrTgQF54AaABAg',
   'comment': "Tous ces anglissimes (sans parler des mots compliqués à traduire afin d'être compris) me donnent envie de m'arracher les oreilles, faites un effort sérieusement.",
   'likeCount': 0,
   'publishedAt': '2024-02-21T12:59:33Z',
   'updatedAt': '2024-02-21T12:59:33Z'},
  'totalReplyCount': 0},
 {'id': 'Ugyfodm1-4R2yZV-jkB4AaABAg',
  'topLevelComment': {'id': 'Ugyfodm1-4R2yZV-jkB4AaABAg',
   'comment': "De toutes façons ce format à l'américaine puant avec des playoffs là faut arrêter.\nLe meilleur format qu'il y a eu en europe c'était le championnat aller retour avec 2 matchs à faire (possibilité de draw du coup) et virer ces playoffs qui sont à la fois incompréhensibles et chiants.",
   'likeCount': 0,
   'publishedAt': '2024-02-21T12:10:02Z',
   'updatedAt': '2024-02-21T12:10:02Z'},
  'totalReplyCount': 0},
 {'id': 'Ugy4SWsZlESMcZYFoUR4AaABAg',
  'topLevelComment': {'id': 'Ugy4SW

# Fetching Top Videos
The goal is to fetch the top 200 videos everyday and to get their comments a week after publishing.

In [56]:
def push_top_vids(topvidsFile:str, regions:list[str], minElapsedTime:int)-> None:
    """ Fetch top 200 vids per region (per <minElapsedTime>) and push in json <topvidsFile> """
    today = datetime.today()
    with open(topvidsFile, 'r') as f:
        data = json.load(f)
        
    if lastUpdate:= data.get('lastUpdate'):
        delta = today - iso_toDatetime(lastUpdate)
        if delta.total_seconds() // 3600 < minElapsedTime:
            raise Exception(f'The fetch request has be done too soon. Next request available in {24-(delta.total_seconds() // 3600)}h ')
        
    data['lastUpdate'] = datetime_toISO(today)
    # Fetching
    for reg in regions:
        if reg not in data.keys():
            data[reg] = {}
        data[reg][datetime_toISO(today)] = get_Most_Popular_Video(youtube, reg)
        
    with open(topvidsFile, 'w') as fichier:
        json.dump(data, fichier)

In [57]:
def create_comment_queue(topvidsFile:str, commentsQueue:str) -> None:
    """ Create a json file <commentsQueue> that stores video IDs which comments have not been fetched """
    with open(topvidsFile, 'r') as f:
        data:dict = json.load(f)
    with open(commentsQueue, 'r') as f:
        queue:dict = json.load(f)   
        
    data.pop('lastUpdate', None)
    date_to_fetch = [dates for dates in list(list(data.values())[0].keys()) if dates not in queue.keys()]
    
    if date_to_fetch: # is empty
        for date in date_to_fetch:  
            comment_list = []
            for region in data.keys():
                # print(f"{region}: {date}. Size {len(data.get(region,{}).get(date,[]))}")
                for video in data.get(region,{}).get(date,[]):
                    comment_list += [
                        {'region': region, 
                        'dateEntry': date, 
                        'id': video.get('id'),
                        'publishedAt': video.get('publishedAt')
                        }
                    ]
            queue[date] = comment_list
        
        with open(commentsQueue, 'w') as fichier:
            json.dump(queue, fichier)    
    else:
        print("already in !")

In [58]:

def fetch_topVids_comments(commentsQueue:str, commentList:str, minElapsedCommentsTime:int) -> None:
    """ Fetches vids comments from <commentsQueue> after minElapsedComments and output them in <commentList>"""
    today = datetime.today()
    with open(commentsQueue, 'r') as f:
        queue:dict[list] = json.load(f)
    with open(commentList, 'r') as f:
        comments:dict[list] = json.load(f)
    
    data={}
    remove_indices:dict[list] = {}
    for date,vids in queue.items():
        for i,video in enumerate(vids):
            if (today - iso_toDatetime(video.get('publishedAt'))).days >= minElapsedCommentsTime: 
                start = time.time()
                data[video.get('id')] = get_video_commentThreads(youtube, video.get('id'), 1000) # crashes a lot
                print(f"{i} in {time.time()-start}s",end='\n\n')
                if date not in remove_indices.keys():
                     remove_indices[date] = []
                remove_indices[date].append(i)
            else:print(f"{i} not yet",end='\n\n')
        
    queue = {date:[video for i, video in enumerate(videos) if i not in remove_indices[date]] for date,videos in queue.items()}  
     
    with open(commentsQueue, 'w') as fichier:
            json.dump(queue, fichier)  
               
    with open(commentList, 'w') as fichier:
            json.dump({**comments, **data}, fichier) # merging data

In [60]:
REGION = ['FR', 'US']
topvids = 'db/topVideos.json'
commentsQueue="db/commentQueue.json"
commentList = "db/commentList.json"
minElapsedTime = 24 # Hours
minElapsedCommentsTime = 17 # days

# push_top_vids(topvids, REGION, minElapsedTime)
    
create_comment_queue(topvids, commentsQueue)
            
# fetch_topVids_comments(commentsQueue, commentList, minElapsedCommentsTime)    